# Applied Data Science Capstone
This notebook will be used for my capstone project in Coursera. 

In [61]:
import pandas as pd 
import numpy as np 
import folium
import requests
from pandas.io.json import json_normalize

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Introduction
Let's imagine that in the past year, my family and I opened a restaurant in Columbus, OH. Specifically, in the Northwest Columbus community. We've achieved a far amount of success and want to expand. Since we know the Northwest community enjoys our food, we should aim to go into a neighborhood that is similar to the Northwest. Through this notebook, we will analyze and cluster and neighborhoods of Columbus to determine potential neighborhoods for our next place of business. 

## Data
The list of Columbus neighborhoods was obtained from [this website](http://opendata.columbus.gov/datasets/c4b483507f374e62bd705450e116e017_25/data). The data here also included the area of each neighborhood in squre feet, which I used to approximate the radius of each neighborhood assuming they are circular. This is not wholly accurate but a good enough approximation. To find the coordinates at the center of each neighborhood, [this map](https://www.arcgis.com/home/webmap/viewer.html?layers=c4b483507f374e62bd705450e116e017), where centers of the neighborhoods were approximated, then the coordinates were copy and pasted into a spreadsheet that was then exported as the [Columbus_Communities.csv](https://github.com/alexanderWhile/Coursera_Capstone/blob/master/Columbus_Communities.csv) found in this repository. 

Let's import our data into a dataframe and preview the information found in it. 

In [62]:
COLUMBUS_COMMUNITIES = pd.read_csv("Columbus_Communities.csv")
COLUMBUS_COMMUNITIES.head()

,Community,Latitude,Longitude,Radius
0,Airport,39.996795,-82.889889,1800
1,Brewery District,39.947067,-83.003872,700
2,Clintonville,40.047406,-83.013828,2200
3,Downtown,39.963515,-82.999752,1400
4,Dublin Road Corridor,39.972330,-83.036144,700


We will start by making a map of the centers of all the communities in Columbus. 

In [63]:
COLUMBUS_LATITUDE = 39.9612
COLUMBUS_LONGITUDE = -82.9988

COLUMBUS_MAP = folium.Map(
    location = [COLUMBUS_LATITUDE, COLUMBUS_LONGITUDE],
    zoom_start = 10,
)

for lat, lng, label in zip(COLUMBUS_COMMUNITIES.Latitude, COLUMBUS_COMMUNITIES.Longitude, COLUMBUS_COMMUNITIES.Community):
    folium.vector_layers.CircleMarker(
        [lat,lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(COLUMBUS_MAP)



COLUMBUS_MAP

*Note: GitHub will not render any folium maps. To see them, follow the link [here](https://nbviewer.jupyter.org/github/alexanderWhile/Coursera_Capstone/blob/master/notebook.ipynb)*

Now let's import our Foursquare credentials to begin utilizing the API and looking up venues. 

In [43]:
CLIENT_ID = 'IA4SDU5HX0UHCL4VSZJDAHBXWJHJY4HPTFNBLWHG4YHYSLWH'
CLIENT_SECRET = '21PID34DCUTLYIWC2RRRRWMBKIE1ZUUXQKE2ZEAASQ4VIWX5'
VERSION = '20200416'

LIMIT = 100

print("Client ID:",CLIENT_ID)
print("Client Secret:", CLIENT_SECRET)
print("Version:", VERSION)
print("Limit:", LIMIT)

Client ID: IA4SDU5HX0UHCL4VSZJDAHBXWJHJY4HPTFNBLWHG4YHYSLWH
Client Secret: 21PID34DCUTLYIWC2RRRRWMBKIE1ZUUXQKE2ZEAASQ4VIWX5
Version: 20200416
Limit: 100


Now we will preview our API calls by making a map of the venues in the community familiar to us, Northwest Columbus. First we will make a folium map centered on the community.

In [44]:
NORTHWEST = COLUMBUS_COMMUNITIES[COLUMBUS_COMMUNITIES.Community == 'Northwest'].reset_index()

NORTHWEST_MAP = folium.Map(
    location = [NORTHWEST.loc[0,'Latitude'], NORTHWEST.loc[0,'Longitude']],
    zoom_start=12
)

folium.vector_layers.CircleMarker(
    [NORTHWEST.loc[0,'Latitude'], NORTHWEST.loc[0,'Longitude']],
    radius=5,
    color = 'red',
    popup = NORTHWEST.loc[0,'Community'],
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(NORTHWEST_MAP)

NORTHWEST_MAP

Next we will make our API call.

In [45]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    NORTHWEST.loc[0,'Latitude'],
    NORTHWEST.loc[0,'Longitude'],
    NORTHWEST.loc[0,'Radius'],
    LIMIT)

results = requests.get(url).json()
print("Success")

Success


Here we define a function to get the category of each venue from the .json file. 

In [46]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

print("Function Defined!")

Function Defined!


We convert the results into a well-formatted data frame.

In [47]:
NORTHWEST_VENUES = results['response']['groups'][0]['items']
NORTHWEST_VENUES = json_normalize(NORTHWEST_VENUES)

FILTERED_COLUMNS = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
NORTHWEST_VENUES = NORTHWEST_VENUES.loc[:,FILTERED_COLUMNS]

NORTHWEST_VENUES['venue.categories'] = NORTHWEST_VENUES.apply(get_category_type,axis = 1)

NORTHWEST_VENUES.columns = [col.split(".")[-1] for col in NORTHWEST_VENUES.columns]

NORTHWEST_VENUES.head()

,name,categories,lat,lng
0,Los Guachos Taqueria,Taco Place,40.064524,-83.057044
1,Graeter's Ice Cream,Ice Cream Shop,40.064990,-83.075559
2,The Grumpy Troll Tavern,Bar,40.064340,-83.060700
3,Somewhere In Particular Brewing,Brewery,40.061978,-83.075634
4,City Egg,Breakfast Spot,40.064127,-83.058756


Check the number of venues.

In [48]:
print("There are", NORTHWEST_VENUES.shape[0], "venues nearby.")

There are 100 venues nearby.


And finally add the venues to our folium map.

In [49]:
for lat, lng, label in zip(NORTHWEST_VENUES.lat, NORTHWEST_VENUES.lng, NORTHWEST_VENUES.name):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'blue',
        popup = label,
    ).add_to(NORTHWEST_MAP)

NORTHWEST_MAP

As a sanity check, we will repeat the process to map all the venues in Downtown Columbus.

In [50]:
DOWNTOWN = COLUMBUS_COMMUNITIES[COLUMBUS_COMMUNITIES.Community == 'Downtown'].reset_index()

DOWNTOWN_MAP = folium.Map(
    location = [DOWNTOWN.loc[0,'Latitude'], DOWNTOWN.loc[0,'Longitude']],
    zoom_start=13
)

folium.vector_layers.CircleMarker(
    [DOWNTOWN.loc[0,'Latitude'], DOWNTOWN.loc[0,'Longitude']],
    radius=5,
    color = 'red',
    popup = DOWNTOWN.loc[0,'Community'],
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(DOWNTOWN_MAP)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    DOWNTOWN.loc[0,'Latitude'],
    DOWNTOWN.loc[0,'Longitude'],
    DOWNTOWN.loc[0,'Radius'],
    LIMIT)

DOWNTOWN_VENUES = results['response']['groups'][0]['items']
DOWNTOWN_VENUES = json_normalize(DOWNTOWN_VENUES)

print(DOWNTOWN_VENUES)

DOWNTOWN_VENUES = DOWNTOWN_VENUES.loc[:,FILTERED_COLUMNS]

DOWNTOWN_VENUES['venue.categories'] = DOWNTOWN_VENUES.apply(get_category_type,axis = 1)

DOWNTOWN_VENUES.columns = [col.split(".")[-1] for col in DOWNTOWN_VENUES.columns]

for lat, lng, label in zip(DOWNTOWN_VENUES.lat, DOWNTOWN_VENUES.lng, DOWNTOWN_VENUES.name):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'blue',
        popup = label,
    ).add_to(DOWNTOWN_MAP)

DOWNTOWN_MAP

referralId  reasons.count  \
0    e-0-4d42eef2715fa143483066c0-0              0   
1    e-0-4b084412f964a520b50723e3-1              0   
2    e-0-4b58ed05f964a520717328e3-2              0   
3    e-0-5bab9abb336273002c845ce2-3              0   
4    e-0-593bfeb7ba4a64672598bd02-4              0   
..                              ...            ...   
95  e-0-4b808228f964a520957830e3-95              0   
96  e-0-4dfa7896d164bbe546171169-96              0   
97  e-0-4e7dab644901ed3061756a86-97              0   
98  e-0-4b07d8b2f964a5206d0023e3-98              0   
99  e-0-4dd9cd201838b8561d227e9f-99              0   

                                        reasons.items  \
0   [{'summary': 'This spot is popular', 'type': '...   
1   [{'summary': 'This spot is popular', 'type': '...   
2   [{'summary': 'This spot is popular', 'type': '...   
3   [{'summary': 'This spot is popular', 'type': '...   
4   [{'summary': 'This spot is popular', 'type': '...   
..                                

In [51]:
def get_nearby_venues(names, latitudes, longitudes, radii):
    venues_list = []
    for name, lat, lng, rad in zip(names, latitudes, longitudes, radii):
        print(name)

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            rad,
            LIMIT)

        results = requests.get(url).json()['response']['groups'][0]['items']

        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

    nearby_venues.columns = [
        'Community',
        'Community Latitude',
        'Community Longitude',
        'Venue',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category']
    
    return(nearby_venues)

print("Success!")

Success!


In [52]:
COLUMBUS_VENUES = get_nearby_venues( 
    names = COLUMBUS_COMMUNITIES.Community,
    latitudes = COLUMBUS_COMMUNITIES.Latitude,
    longitudes = COLUMBUS_COMMUNITIES.Longitude,
    radii = COLUMBUS_COMMUNITIES.Radius
)

Airport
Brewery District
Clintonville
Downtown
Dublin Road Corridor
East Columbus
Far East
Far North
Far Northwest
Far South
Far West
Fifth by Northwest
Fort Hayes
Franklinton
German Village
Greater Hilltop
Harmon Road Corridor
Harrison West
Hayden Run
Italian Village
Livingston Avenue Area
Mid East
Milo-Grogan
Near East
North Central
North Linden
Northeast
Northland
Northwest
Olentangy West
Rocky Fork-Blacklick
South East
South Linden
South Side
Southwest
State of Ohio
University District
Victorian Village
West Scioto
Westland
Wolfe Park


In [53]:
print(COLUMBUS_VENUES.shape)
COLUMBUS_VENUES

(2368, 7)


,Community,Community Latitude,Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Airport,39.996795,-82.889889,Fairfield Inn & Suites Columbus Airport,39.999001,-82.890372,Hotel
1,Airport,39.996795,-82.889889,CMH Passenger Drop-off / Pick-up,39.998053,-82.884504,Airport Service
2,Airport,39.996795,-82.889889,Southwest Airlines Ticket Counter,39.997923,-82.884274,Airport Service
3,Airport,39.996795,-82.889889,Starbucks,39.997821,-82.881962,Coffee Shop
4,Airport,39.996795,-82.889889,Enterprise Rent-A-Car,39.998289,-82.887247,Rental Car Location
...,...,...,...,...,...,...,...
2363,Westland,39.922398,-83.145575,Wendy’s,39.951000,-83.147086,Fast Food Restaurant
2364,Westland,39.922398,-83.145575,Prairie Township Road Department,39.905203,-83.174205,Construction & Landscaping
2365,Westland,39.922398,-83.145575,La Super Torta,39.938141,-83.113514,Mexican Restaurant
2366,Westland,39.922398,-83.145575,Liberty Freight,39.951690,-83.148999,Furniture / Home Store


In [54]:
COLUMBUS_VENUES.groupby('Community').count()

,Community Latitude,Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Community,,,,,,
Airport,49,49,49,49,49,49
Brewery District,38,38,38,38,38,38
Clintonville,100,100,100,100,100,100
Downtown,96,96,96,96,96,96
Dublin Road Corridor,11,11,11,11,11,11
East Columbus,14,14,14,14,14,14
Far East,100,100,100,100,100,100
Far North,100,100,100,100,100,100
Far Northwest,37,37,37,37,37,37


In [55]:
print('There are {} unique categories'.format(len(COLUMBUS_VENUES['Venue Category'].unique())))

There are 273 unique categories


In [56]:
COLUMBUS_ONEHOT = pd.get_dummies(COLUMBUS_VENUES[['Venue Category']],prefix="",prefix_sep="")

COLUMBUS_ONEHOT['Community'] = COLUMBUS_VENUES['Community']

cols = list(COLUMBUS_ONEHOT)
cols.insert(0,cols.pop(cols.index('Community')))

COLUMBUS_ONEHOT = COLUMBUS_ONEHOT.loc[:,cols]
COLUMBUS_ONEHOT.head()

,Community,ATM,Accessories Store,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Airport,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Airport,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
COLUMBUS_GROUPED = COLUMBUS_ONEHOT.groupby('Community').mean().reset_index()

COLUMBUS_GROUPED

,Community,ATM,Accessories Store,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Airport,0.000000,0.000000,0.00,0.020408,0.122449,0.020408,0.061224,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brewery District,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.026316,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Clintonville,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,...,0.01,0.00,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000
3,Downtown,0.010417,0.000000,0.00,0.000000,0.000000,0.000000,0.052083,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Dublin Road Corridor,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.090909,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909
5,East Columbus,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.071429,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Far East,0.020000,0.000000,0.00,0.000000,0.000000,0.000000,0.060000,0.00,0.010000,...,0.01,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000
7,Far North,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.080000,0.00,0.000000,...,0.00,0.01,0.00,0.000000,0.010000,0.010000,0.000000,0.010000,0.000000,0.000000
8,Far Northwest,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Far South,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.070000,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000


In [67]:
num_top_venues = 5

for comm in COLUMBUS_GROUPED.Community:
    print("---"+comm+"---")
    temp = COLUMBUS_GROUPED[COLUMBUS_GROUPED.Community == comm].T.reset_index()
    temp.columns = ['venue','percent']
    temp = temp.iloc[1:]
    temp['percent'] = temp['percent'].astype(float)*100
    temp = temp.round({'percent':1})
    print(temp.sort_values('percent',ascending = False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Airport---
                 venue  percent
0  Rental Car Location     14.3
1      Airport Service     12.2
2                Hotel      8.2
3  American Restaurant      6.1
4          Coffee Shop      6.1


---Brewery District---
                venue  percent
0                 Bar     10.5
1                Bank      5.3
2          Sports Bar      5.3
3  Athletics & Sports      5.3
4             Brewery      5.3


---Clintonville---
            venue  percent
0     Coffee Shop      5.0
1  Sandwich Place      4.0
2            Bank      4.0
3          Bakery      4.0
4     Pizza Place      4.0


---Downtown---
                 venue  percent
0  American Restaurant      5.2
1          Pizza Place      5.2
2                 Café      4.2
3              Brewery      4.2
4              Theater      4.2


---Dublin Road Corridor---
                    venue  percent
0  Furniture / Home Store     18.2
1             Yoga Studio      9.1
2          Hardware Store      9.1
3      Salon / Barbers